# Example: Comparison of $\Pi_c$

In [3]:
from adlerpy.adler_sm import adler_charm_pert
from adlerpy.adler_sm import adler_bottom_pert
from scipy.optimize import fsolve

from adlerpy.adler_routines import Particle
from adlerpy.adler_routines import adler_massless_connected
from scipy.integrate import quad
from matplotlib import pyplot as plt 
import numpy as np 
import pandas as pd


#Create the particles
aZ=0.1185;
Mz=91.1876;


#Here you define the SM particles 

up=Particle("up",x=[2.16*0.001,2,3],mudec=0.001,mpole=None,mpole_on=False)
down=Particle("down",x=[4.67*0.001,2,3],mudec=0.001,mpole=None,mpole_on=False)
strange=Particle("strange",x=[0.09203,2,3],mudec=0.001,mpole=None,mpole_on=False)
charm=Particle("charm",x=[1.273,1.273,4],mudec=2*1.273,mpole=1.67,mpole_on=False)
bottom=Particle("bottom",x=[4.18,4.18,5],mudec=2*4.18,mpole=4.25,mpole_on=False)
top=Particle("top",x=[164,164,6],mudec=164,mpole=164,mpole_on=False)
particle_list=[up,charm,down,strange,bottom,top] 


## Load lattice points

In [ ]:
Πcc_df=pd.read_csv('Picc.csv')
Q2_list=np.array(Πcc_df['Q2GeV'][0:80])
Pilatc=4/9*np.array(Πcc_df['$\Pi_{cc}$'][0:80]) #I must to add the charge square to the lattice value
error=4/9*np.array(Πcc_df["total"][0:80])

nloops_list=[0,1,2,3] #nloops to be compared

#----For plotting later:
loops_style=["dotted","-.","--","solid"] 
labels_MS=["$\mathcal{O}(\hat{\\alpha}^{0}_s)$ $\overline{\mathrm{MS}}$",
          "$\mathcal{O}(\hat{\\alpha}^{1}_s)$ $\overline{\mathrm{MS}}$",
          "$\mathcal{O}(\hat{\\alpha}^{2}_s)$$\overline{\mathrm{MS}}$",
          "$\mathcal{O}(\hat{\\alpha}^{3}_s)$ $\overline{\mathrm{MS}}$",]   
labels_OS=["$\mathcal{O}(\hat{\\alpha}^{0}_s)$ OS",
           "$\mathcal{O}(\hat{\\alpha}^{1}_s)$ OS",
          "$\mathcal{O}(\hat{\\alpha}^{2}_s)$ OS",
          "$\mathcal{O}(\hat{\\alpha}^{3}_s)$ OS"]   


## Define the intgral to get $\Pi_c$ from $D_c$

In [ ]:
# Compare with lattice QCD:

def integrand(Q2,aZ,particle_list,mpole_on,nloops,cut_high_as3,cut_low_as3,GG):
    alpha0=1/137.036
    Q=np.sqrt(Q2);
    return 1/12/np.pi**2*(adler_charm_pert(aZ,Mz,Q=Q,particles=particle_list,mpole_on=mpole_on,nloops=nloops,
                                           cut_high_as3=cut_high_as3,cut_low_as3=cut_low_as3,GG=GG))/Q2;

def Pi(Q2,aZ,particle_list,mpole_on,nloops,cut_high_as3=20,cut_low_as3=0.3,GG=0):
    return quad(integrand, 0.0000, Q2,args=(aZ,particle_list,mpole_on,nloops,cut_high_as3,cut_low_as3,GG))[0]


## Compute $\Pi_c$

In [ ]:

PiMS_list=[]
PiOS_list=[]
for nloops in nloops_list:
    pims=[]
    pios=[]
    print("I am computing the ",nloops, "order")
    for Q2 in Q2_list:
        pims.append(Pi(Q2,aZ=aZ,particle_list=particle_list,mpole_on=False,nloops=nloops,cut_high_as3=20,cut_low_as3=1))
        pios.append(Pi(Q2,aZ=aZ,particle_list=particle_list,mpole_on=True,nloops=nloops,cut_high_as3=20,cut_low_as3=1))
    PiMS_list.append(pims)
    PiOS_list.append(pios)


## Lets make the plots

In [ ]:
fig, ax = plt.subplots()

plt.scatter(Q2_list, Pilatc, color="Black",s=0.2)
plt.errorbar(Q2_list, Pilatc, yerr=error,capsize=1, ls = "None",elinewidth=0.5, capthick=0.5,label='Lattice Mainz',color="Black")
for i in range(len(nloops_list)):
    plt.plot(Q2_list, PiMS_list[i],color="blue",linestyle=loops_style[i],label=labels_MS[i],linewidth=0.5)
    plt.plot(Q2_list, PiOS_list[i],color="red",linestyle=loops_style[i],label=labels_OS[i],linewidth=0.5)

plt.xlabel('$Q^2$ (GeV)',fontsize=16)
plt.ylabel('$\Pi_c(Q)$',fontsize=17)

#ax.set(xlabel='$Q^2$ (GeV)', ylabel='$\Pi_c(Q)$',fontsize=16)
plt.grid()
plt.xticks(fontsize=14)
ax.set_xlim([0.5,3])
ax.set_ylim([0,0.006])
ax.spines['bottom'].set_linewidth(1.3)
ax.spines['left'].set_linewidth(1.3)
ax.spines['top'].set_linewidth(1.3)
ax.spines['right'].set_linewidth(1.3)
ax.legend(bbox_to_anchor=(1.03, 0.9), loc='upper left',prop={'size': 10})

fig.savefig("ExampleVacuumPolcharm.png",dpi=900,bbox_inches='tight')
plt.show()

## Compute corresponding mass
Let us now compute to which value of the $\overline{\mathrm{MS}}$ the lattice result corresponds to. 

In [ ]:
# Compare with lattice QCD:

def Pi_c(mc,Q2,aZ,nloops,GG=0):
    charm_internal=Particle("charm",x=[mc,mc,4],mudec=2*mc,mpole=1.67,mpole_on=False)
    particle_list=[up,charm_internal,down,strange,bottom,top] 
    return  Pi(Q2,aZ,particle_list,mpole_on=False,nloops=nloops,cut_high_as3=20,cut_low_as3=1,GG=GG)
   

The $\hat{m}_c$ value is very stable. Let us take i=38, which correspondes to $Q^2=0.5\,\mathrm{GeV}$
as our reference point. 

In [ ]:
i=38
aZ=0.1185
daZ=0.0008
q2=Q2_list[i];
pilat=Pilatc[i];
nloops=3;

func = lambda mc :pilat - Pi_c(float(mc),q2,aZ,nloops)
mc = fsolve(func,1.27)[0]


Compute the error from lattice

In [ ]:
func = lambda mc :pilat+error[i] - Pi_c(float(mc),q2,aZ,nloops)
sol = fsolve(func,1.27)[0]
laterr=(sol-mc) 
print(laterr)

Compute the parametric dependence on $\alpha_s$ 

In [ ]:
func = lambda mc :pilat - Pi_c(float(mc),q2,aZ+daZ,nloops)
sol = fsolve(func,1.27)[0]
paramas=(sol-mc)/daZ
print(paramas)

Compute the truncation error 

In [ ]:
func = lambda mc :pilat - Pi_c(float(mc),q2,aZ,nloops-1)
sol = fsolve(func,1.27)[0]
trunc=(sol-mc)
print(trunc)

Compute condensate effects

In [ ]:
func = lambda mc :pilat - Pi_c(float(mc),q2,aZ,nloops,GG=0.01)
asol = fsolve(func,1.27)[0]
cond=(asol-mc)
print(cond)

Pretty printing

In [ ]:
text=("\hat{m}_c(\hat{m}_c)= "+ str(round(mc,3))+
      "+"+
      str(round(paramas,1))+
      "(\hat{\\alpha}_s-0.1185)"+
      "\pm"+
      str(round(np.abs(laterr),3))+
      "_{\mathrm{latt}} \pm"+
      str(round(np.abs(trunc),3))+"_{\mathrm{tr}}\pm"
      +
      str(round(np.abs(cond),3))+"_{\mathrm{cond}}"
     )
from IPython.display import display, Math
display(Math(text))
dmc=np.sqrt((paramas*0.0016)**2+laterr**2+trunc**2+cond**2)
print("total_error=",round(dmc,3))

## Make plot to compare with other results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# example data
x = np.arange(0, 4, 1)
y = -0.2* x

labels=["Mainz:2022 (in this work)","Petreczky:19","Erler:2016","Maezawa:2016",
        "Nakayama:2016","Chakraborty:2014","Dehnadi:2015","Narison:2011","Bodenstein:2011","Chetyrkin:2010","HPQCD 10"]
Mw_list=[mc,1.265,1.274,1.267,1.2871,1.2715,1.288,1.261,1.278,1.279,1.273]
σ_list=[dmc,0.010,0.009,0.011,0.01,0.0095,0.020,0.016,0.009,0.013,0.006]
y_list=np.linspace(-9,9,num=len(σ_list))

lower_error =  σ_list
upper_error =  σ_list
asymmetric_error = np.array(list(zip(lower_error, upper_error))).T

fig = plt.figure()
ax = fig.gca()
plt.grid()
ax.axes.get_yaxis().set_visible(False)
plt.xticks(fontsize=14)
plt.xlabel('$\hat{m}_c(\hat{m}_c)$ GeV ',fontsize=16)

for i in range(len(Mw_list)):
    plt.text(1.185, y_list[i], labels[i],dict(size=12))
plt.errorbar(Mw_list, y_list, xerr=asymmetric_error, fmt='o',markersize=5,color="purple",capsize=3.5, capthick=1.5,elinewidth=1.9,ecolor = 'Black')


ax.spines['bottom'].set_linewidth(1.9)
ax.spines['left'].set_linewidth(1.9)
ax.spines['top'].set_linewidth(1.9)
ax.spines['right'].set_linewidth(1.9)

ax.set_xlim([1.23,1.32])
ax.set_ylim([-10,10])
ax.grid(which='major', color='#DDDDDD', linewidth=2.2)
ax.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=2.2)
ax.minorticks_on()
plt.savefig('mcfromlattice.png',dpi=500,bbox_inches='tight')
plt.show()
